In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3,2"

import transformers
import torch
from transformers import  AutoTokenizer,AutoModelForCausalLM, GenerationConfig, pipeline
from peft import PeftModel

model_id = "models/Base/Qwen-7B-Chat"
model_id = "models/Lora/Qwen_7B_Chat_lora_step1"
model_id = "models/Lora/Qwen_7B_Chat_lora_step2"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    load_in_4bit = True,
    use_cache_quantization=True,
    use_cache_kernel=True, 
    trust_remote_code=True,
    use_flash_attn = False,
).eval()
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=False, 
    trust_remote_code=True
)


/home/newdisk/jxh/anaconda/envs/sft2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
/home/jxh/.cache/huggingface/modules/transformers_modules/Qwen-7B-Chat/modeling_qwen.py:314: UserWarning: KV cache kernel source files (.cpp and .cu) not found.
  warnings.warn("KV cache kernel source files (.cpp and .cu) not found.")
Loading checkpoint shards: 100%|██████████| 8/8 [00:04<00:00,  1.95it/s]


In [2]:
model.enable_adapters()

In [38]:
model.disable_adapters()

In [3]:
model.generation_config.temperature = 0.4
model.generation_config.max_new_tokens = 256
model.generation_config.top_p = 0.7

model.generation_config


GenerationConfig {
  "chat_format": "chatml",
  "do_sample": true,
  "eos_token_id": 151643,
  "max_new_tokens": 256,
  "max_window_size": 6144,
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.4,
  "top_k": 0,
  "top_p": 0.7
}

In [4]:
data = [
     "ST星源2022年12月2日在深交所互动易中披露，截至2022年11月30日公司股东户数为8.99万户，较上期（2022年9月30日）减少447户，减幅为0.49%。ST星源股东户数高于行业平均水平。根据Choice数据，截至2022年11月30日环保行业上市公司平均股东户数为2.95万户。其中，公司股东户数处于0.5万~1.5万区间占比最高，为26.40%。环保行业股东户数分布股东户数与股价2022年6月30日至今，公司股东户数有所下降，区间跌幅为3.17%。2022年6月30日至2022年11月30日区间股价上涨3.85%。股东户数及股价股东户数与股本截至2022年11月30日，公司最新总股本为10.59亿股，其中流通股本为10.58亿股。户均持有流通股数量由上期的1.17万股上升至1.18万股，户均流通市值2.22万元。户均持股金额",
     "平安银行总经理内部讲话公开，要相信公司前景",
]

system_prompt = "你是一个中文文本分类器，判断以下金融文本的情绪类别，答案为积极、消极或者中性。"

response, history = model.chat(tokenizer, data[0], history = None, system = system_prompt)
print(response)

该文本情感是中性的。


In [16]:
model.disable_adapters()
system_prompt = "你是一个中文文本分类器，判断以下金融文本表达情绪的显隐性，答案为显性或隐性。直抒胸臆，直接表达看法为主则为显性；通过数据来支持或暗示观点为隐性。"

response, history = model.chat(tokenizer, data[1], history = None, system = system_prompt)
print(response)

显性
